index

gender: 성별

car: 차량 소유 여부

reality: 부동산 소유 여부

child_num: 자녀 수

income_total: 연간 소득


income_type: 소득 분류
							['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']
edu_type: 교육 수준
							['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']

family_type: 결혼 여부
							['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']

house_type: 생활 방식
							['Municipal apartment', 'House / apartment', 'With parents',

							 'Co-op apartment', 'Rented apartment', 'Office apartment']

DAYS_BIRTH: 출생일


							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미


DAYS_EMPLOYED: 업무 시작일

							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미
							양수 값은 고용되지 않은 상태를 의미함

FLAG_MOBIL: 핸드폰 소유 여부

work_phone: 업무용 전화 소유 여부

phone: 전화 소유 여부

email: 이메일 소유 여부

occyp_type: 직업 유형													

family_size: 가족 규모

begin_month: 신용카드 발급 월
							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미

credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도


							=> 낮을 수록 높은 신용의 신용카드 사용자를 의미함

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedKFold

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from kaggler.model import AutoLGB
from imblearn.over_sampling import SMOTE

In [2]:
train = pd.read_csv('train.csv', index_col='index')
test = pd.read_csv('test.csv', index_col='index')

In [3]:
train.shape

(26457, 19)

In [4]:
train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,,
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26457 entries, 0 to 26456
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         26457 non-null  object 
 1   car            26457 non-null  object 
 2   reality        26457 non-null  object 
 3   child_num      26457 non-null  int64  
 4   income_total   26457 non-null  float64
 5   income_type    26457 non-null  object 
 6   edu_type       26457 non-null  object 
 7   family_type    26457 non-null  object 
 8   house_type     26457 non-null  object 
 9   DAYS_BIRTH     26457 non-null  int64  
 10  DAYS_EMPLOYED  26457 non-null  int64  
 11  FLAG_MOBIL     26457 non-null  int64  
 12  work_phone     26457 non-null  int64  
 13  phone          26457 non-null  int64  
 14  email          26457 non-null  int64  
 15  occyp_type     18286 non-null  object 
 16  family_size    26457 non-null  float64
 17  begin_month    26457 non-null  float64
 18  credit

In [6]:
total = train.isnull().sum().sort_values(ascending=False)
total2 = test.isnull().sum().sort_values(ascending=False)
print(total)

occyp_type       8171
gender              0
DAYS_EMPLOYED       0
begin_month         0
family_size         0
email               0
phone               0
work_phone          0
FLAG_MOBIL          0
DAYS_BIRTH          0
car                 0
house_type          0
family_type         0
edu_type            0
income_type         0
income_total        0
child_num           0
reality             0
credit              0
dtype: int64


In [7]:
train.drop(['FLAG_MOBIL'], axis=1, inplace=True)
train.fillna('BaekSu', inplace=True) 

test.drop(['FLAG_MOBIL'], axis=1, inplace=True)
test.fillna('BaekSu', inplace=True)


In [8]:
feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
for feat in feats:
    train[feat]=-train[feat]  
    test[feat]=-test[feat]    


In [9]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
numerical_feats.remove('credit')
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

total_feats = numerical_feats + categorical_feats

Number of Numerical features:  9
Number of Categorical features:  8


### ID생성 / 중복컬럼 제거

In [10]:
train['ID'] = \
train['child_num'].astype(str) + '_' + train['income_total'].astype(str) + '_' +\
train['DAYS_BIRTH'].astype(str) + '_' + train['DAYS_EMPLOYED'].astype(str) + '_' +\
train['work_phone'].astype(str) + '_' + train['phone'].astype(str) + '_' +\
train['email'].astype(str) + '_' + train['family_size'].astype(str) + '_' +\
train['gender'].astype(str) + '_' + train['car'].astype(str) + '_' +\
train['reality'].astype(str) + '_' + train['income_type'].astype(str) + '_' +\
train['edu_type'].astype(str) + '_' + train['family_type'].astype(str) + '_' +\
train['house_type'].astype(str) + '_' + train['occyp_type'].astype(str)

# test['ID'] = \
# test['child_num'].astype(str) + '_' + test['income_total'].astype(str) + '_' +\
# test['DAYS_BIRTH'].astype(str) + '_' + test['DAYS_EMPLOYED'].astype(str) + '_' +\
# test['work_phone'].astype(str) + '_' + test['phone'].astype(str) + '_' +\
# test['email'].astype(str) + '_' + test['family_size'].astype(str) + '_' +\
# test['gender'].astype(str) + '_' + test['car'].astype(str) + '_' +\
# test['reality'].astype(str) + '_' + test['income_type'].astype(str) + '_' +\
# test['edu_type'].astype(str) + '_' + test['family_type'].astype(str) + '_' +\
# test['house_type'].astype(str) + '_' + test['occyp_type'].astype(str)

In [11]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit,ID
index,,,,,,,,,,,,,,,,,,,
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,0,0,0,BaekSu,2.0,6.0,1.0,0_202500.0_13899_4709_0_0_0_2.0_F_N_N_Commerci...
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,0,0,1,Laborers,3.0,5.0,1.0,1_247500.0_11380_1540_0_0_1_3.0_F_N_Y_Commerci...
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,19087,4434,0,1,0,Managers,2.0,22.0,2.0,0_450000.0_19087_4434_0_1_0_2.0_M_Y_Y_Working_...
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,0,1,0,Sales staff,2.0,37.0,0.0,0_202500.0_15088_2092_0_1_0_2.0_F_N_Y_Commerci...
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,15037,2105,0,0,0,Managers,2.0,26.0,2.0,0_157500.0_15037_2105_0_0_0_2.0_F_Y_Y_State se...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,12079,1984,0,0,0,Core staff,4.0,2.0,1.0,2_225000.0_12079_1984_0_0_0_4.0_F_N_N_State se...
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,15291,2475,0,0,0,BaekSu,2.0,47.0,2.0,1_180000.0_15291_2475_0_0_0_2.0_F_N_Y_Working_...
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,10082,2015,0,0,0,Core staff,2.0,25.0,2.0,0_292500.0_10082_2015_0_0_0_2.0_F_Y_N_Working_...


In [12]:
train = train.sort_values(by=['begin_month'], axis=0).drop_duplicates(['ID'], keep='first')
#test = test.sort_values(by=['begin_month'], axis=0).drop_duplicates(['ID'], keep='first')

In [13]:
train.sort_values(by=['index'], axis=0, inplace=True, ignore_index=True)
#test.sort_values(by=['index'], axis=0, inplace=True, ignore_index=True)

In [14]:
train.drop(['ID'], axis=1, inplace=True)
#test.drop(['ID'], axis=1, inplace=True)

In [15]:
#new_data=pd.DataFrame(train.groupby(["ID"])["begin_month"].agg(min))
#new_data

## LabelEncoding, Scaling

In [16]:
for feat in categorical_feats:
    encoder = LabelEncoder()
    train[feat] = encoder.fit_transform(train[feat])
    test[feat] = encoder.transform(test[feat])

In [17]:
for col in ['child_num', 'family_size']:
    train[col] = np.log2(1 + train[col])
    test[col] = np.log2(1 + test[col])
    
train['DAYS_BIRTH'] = np.log2(1 + train['DAYS_BIRTH'])
test['DAYS_BIRTH'] = np.log2(1 + test['DAYS_BIRTH'])

scaler = StandardScaler()
train[numerical_feats] = scaler.fit_transform(train[numerical_feats])
test[numerical_feats] = scaler.transform(test[numerical_feats])

### 중복이슈

In [ ]:
train[train[train.columns].duplicated(keep = False)].sort_values(["DAYS_BIRTH","income_total"])

In [ ]:
df1 = train[train.drop(["begin_month"], axis = 1).duplicated(keep = False)].sort_values(["DAYS_BIRTH","income_total"])

In [ ]:
df1.head(50)

In [ ]:
train.drop(["begin_month"], axis = 1).duplicated().sum()

### 직업은 없지만 일을 하고 있는 사람이 있다

In [ ]:
#train[df_train['DAYS_EMPLOYED'] == 365243]

In [ ]:
#train[df_train['occyp_type'] == 'BaekSu']

In [ ]:
# enc = OneHotEncoder()
# enc.fit(df_train.loc[:,categorical_feats])

# df_train_onehot = pd.DataFrame(enc.transform(df_train.loc[:,categorical_feats]).toarray(), 
#              columns=enc.get_feature_names(categorical_feats))
# df_train.drop(categorical_feats, axis=1, inplace=True)
# df_train = pd.concat([df_train, df_train_onehot], axis=1)

# df_train_onehot = pd.DataFrame(enc.transform(df_test.loc[:,categorical_feats]).toarray(), 
#              columns=enc.get_feature_names(categorical_feats))
# df_test.drop(categorical_feats, axis=1, inplace=True)
# df_test = pd.concat([df_test, df_train_onehot], axis=1)

## Modeling

In [18]:
n_est = 1000
seed = 42
n_fold = 5
n_class = 3

rf_params = {
    'max_depth': 15,
    'min_samples_leaf': 8,
    'random_state': seed
}

lgb_params = {
    'metric': 'multi_logloss',
    'n_estimators': n_est,
    'objective': 'multiclass',
    'random_state': seed,
    'learning_rate': 0.01,
    'min_child_samples': 20,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 64,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'num_class': n_class
}

xgb_params = {
    'objective': 'multi:softprob',
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'learning_rate': 0.01,
    'random_state': seed,
    'num_class': n_class,
    'max_depth': 6,
    'n_estimators': n_est,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
}

In [19]:
models = {'RF': RandomForestClassifier(**rf_params), 
               'LGBM': LGBMClassifier(**lgb_params),
               'XGB': XGBClassifier()}

In [20]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
X = X_balanced
y = y_balanced
X_test = test[total_feats]

xgb = XGBClassifier()
xgb.fit(X, y)
xgb_pred = xgb.predict_proba(X_test)

In [ ]:
y = train['credit']
print(f'\tCV Log Loss: {log_loss(y, xgb_pred):.6f}')

In [ ]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

pred_dic = {}
pred_test_dic = {}
        
for name in models:
    print(f'Training {name}:')
    pred = np.zeros((X.shape[0], 3), dtype=float)
    pred_test = np.zeros((X_test.shape[0], 3), dtype=float)
    for fold in range(5):
        train_idx, valid_idx = folds[fold]
        X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
        models[name].fit(X_train, y_train)
        pred[valid_idx] = models[name].predict_proba(X_valid)
        pred_test += models[name].predict_proba(X_test) / 5
        
    pred_dic[name] = pred
    pred_test_dic[name] = pred_test
    print(f'\tCV Log Loss: {log_loss(y, pred):.6f}')

In [ ]:
pred_df = pd.DataFrame(data=xgb_pred)
pred_df

In [ ]:
sub = pd.read_csv('sample_submission.csv', index_col='index')
sub[sub.columns] = pred_df.values
sub.to_csv('submission0427_3.csv')

### Stacking - Failed
ref) https://www.dacon.io/competitions/official/235713/codeshare/2594?page=1&dtype=recent

In [ ]:
# X = pd.DataFrame(np.hstack([x for _, x in pred_dic.items()]))
# X_test = pd.DataFrame(np.hstack([x for _, x in pred_test_dic.items()]))

# pred = np.zeros((X.shape[0], 3), dtype=float)
# pred_test = np.zeros((X_test.shape[0], 3), dtype=float)
# for fold in range(5):
#     if fold == 0:
#         train_idx, valid_idx = folds[fold]
#         X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
        
#         model = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': n_class}, 
#         feature_selection=False, n_est=10000)
#         model.tune(X_train, y_train)
#         n_best = model.n_best
#         features = model.features
#         params = model.params
        
#         print(f'best iteration: {n_best}')
#         print(f'selected features ({len(features)}): {features}')      
#         print(params)
#         model.fit(X_train, y_train)
#     else:
#         train_data = lgb.Dataset(X[features].iloc[train_idx], label=y[train_idx])
#         model = lgb.train(params, train_data, n_best, verbose_eval=100)
    
#     pred[valid_idx] = models[name].predict_proba(X_valid)
#     pred_test += models[name].predict_proba(X_test) / 5
    